In [1]:
!kubectl create -f deployments/mab-model.yaml

seldondeployment.machinelearning.seldon.io/mab-model created


In [2]:
!cat deployments/mab-model.yaml

apiVersion: machinelearning.seldon.io/v1alpha3
kind: SeldonDeployment
metadata:
  generation: 1
  labels:
    app: seldon
  name: mab-model
spec:
  name: mab-model
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - env:
          - name: PREDICTIVE_UNIT_SERVICE_PORT
            value: "9000"
          - name: PREDICTIVE_UNIT_ID
            value: classifier-1
          - name: PREDICTOR_ID
            value: abtest
          - name: SELDON_DEPLOYMENT_ID
            value: mab-model
          image: pipelineai/predict-meanclassifier:2.0.0 
          imagePullPolicy: IfNotPresent
          lifecycle:
            preStop:
              exec:
                command:
                - /bin/sh
                - -c
                - /bin/sleep 10
          livenessProbe:
            failureThreshold: 3
            initialDelaySeconds: 60
            periodSeconds: 5
            successThreshold: 1
            tcpSocket:
              port: http
            timeoutSec

# Predict
Notes:
* Change the `<ip-address>` below with the ip/hostname in your browser url
* If you see `<!DOCTYPE html>` or `upstream error`, please try again in a bit
* The model servers may take 30-60 seconds to start, so please be patient

In [65]:
%%bash

curl \
  https://community.cloud.pipeline.ai/seldon/kubeflow/mab-model/api/v0.1/predictions \
  -d '{"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}' \
  -H "Content-Type: application/json"


{
  "code": 203,
  "info": "org.springframework.web.client.HttpClientErrorException: 400 BAD REQUEST",
  "reason": "Microservice error",
  "status": "FAILURE"
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   232  100   160  100    72    313    141 --:--:-- --:--:-- --:--:--   313


# Feedback / Reward

In [64]:
%%bash

curl \
  http://mab-model-abtest-eg-router-seldonio-mab-epsilon-greedy-1-3:9000/send-feedback \
  -d '{"request":{"data":{"names":["a","b"],"tensor":{"shape":[1,2],"values":[0,1]}}},"response":{"data":{"names":["a","b"],"tensor":{"shape":[1,1],"values":[0.9]}}},"reward":1}' \
  -H "Content-Type: application/json"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:02:10 --:--:--     0curl: (7) Failed to connect to mab-model-abtest-eg-router-seldonio-mab-epsilon-greedy-1-3 port 9000: Connection timed out


CalledProcessError: Command 'b'\ncurl \\\n  http://mab-model-abtest-eg-router-seldonio-mab-epsilon-greedy-1-3:9000/send-feedback \\\n  -d \'{"request":{"data":{"names":["a","b"],"tensor":{"shape":[1,2],"values":[0,1]}}},"response":{"data":{"names":["a","b"],"tensor":{"shape":[1,1],"values":[0.9]}}},"reward":1}\' \\\n  -H "Content-Type: application/json"\n'' returned non-zero exit status 7.

# Current Feedback for a Given Route

In [ ]:
%%bash

curl \
  http://mab-model-abtest-eg-router-seldonio-mab-epsilon-greedy-1-3:9000/route \
  -d '{"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}' \
  -H "Content-Type: application/json"
